In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder 
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
import string
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/xenang/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [22]:
#combine Excel
hosp_lists = os.listdir("OCR")
hospitalname = []

for i in hosp_lists:
    sep = i.split()
    sep2 = sep[3].split(".")
    hospitalname.append(sep2[0])
hospitalname


combined_df = pd.DataFrame()

for hospitals in hosp_lists:
    excel_file_path = "OCR/" + hospitals

    all_sheets = pd.read_excel(excel_file_path, sheet_name=None)
    sep = hospitals.split()
    sep2 = sep[3].split(".")
    hospitalname = sep2[0]


    for sheet_name, sheet_df in all_sheets.items():
        sheet_df['Codes'] = sheet_name
        sheet_df['Hospitals'] = hospitalname
        try:
            combined_df = pd.concat([combined_df, sheet_df], ignore_index=True)
        except ValueError as e:
            print(f"Error processing sheet '{sheet_name}': {e}")


In [24]:
combined_df.dropna(subset=['Date'], inplace=True)

KeyError: ['Date']

In [ ]:
#prepare for NLP
def concat_columns(row):
    return ' '.join(str(item) for item in row)

combined_df['NLP'] = combined_df[['Provider', 'Type', 'Heading','Description','Category','LOG Dx']].apply(concat_columns, axis=1)

descp = combined_df['NLP'].tolist()
unique_desc = list(set(descp))

cat = []

for i in unique_desc:
    s = i.split()
    if s[0] not in cat:
        cat.append(s[0])

In [ ]:
get_excel_file = 'Input/cancer_list.xlsx'
cancer_list = pd.read_excel(get_excel_file)
cancer_word_list = []

for cancer_type in cancer_list['Cancer type'].dropna():
    if cancer_type not in cancer_word_list:
        cancer_word_list.append(cancer_type)

In [ ]:
keywords =['cancer','chemo','radiotherapy','endocrine','hormonal','immunotherapy','target','biopsy',"immunotherapy", "metastasis", "benign",
            "Biopsy forceps", "Gastric Biopsy","viopsy", "Biopsy Forcep" ,"Disposable Biopsy forceps" ,'Core Biopsy','Radiosurgery','Oncology', "LABORATORY ",
            'Chemotherapy','core','specimen','leukemia' , 'leukaemia' , "filgrastim", "tumour","tumors","tumor", "cytology", "chemotherapy-induced", "Ondansetron"
            ]

keyword = keywords + cancer_word_list
#len(keyword)


In [ ]:

lowercase = [str(item).lower() if isinstance(item, str) else item for item in keyword]

lowercase_list = lowercase
"""
lowercase_list = []
for i in lowercase:
    splits = str(i).split()
    if len(splits) > 1:
        for j in splits:
            lowercase_list.append(j)
    else:
        lowercase_list.append(i)

"""

common_words = {'a','b', 'and', 'or', 'the','for', 'with', 'not', 'by', 'ii','iii','system','is', 'of', 
                'in', 'on', 'at', 'to','1','2','3','4','5','6','7','8','9','12','surgery','1st','2a','3b','complex',
                'c','t3','t4','treatment','type','during','only','core','exam','screening'}

# Input list

# Remove common words from the input list
lowercase_list2 = [word for word in lowercase_list if word not in common_words]

def remove_duplicates(lst):
    seen = set()
    result = []
    for item in lst:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result


unique_list = remove_duplicates(lowercase_list2)


In [ ]:
if "colorectal cancer" in unique_list:
    print(True)

True


In [ ]:
def remove_punctuation(text):
    return ''.join([char for char in text if char not in string.punctuation])
porter = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def process_text(text):
    tokens = word_tokenize(text)
    tokens = [remove_punctuation(token.lower()) for token in tokens if token.isalnum()]  # Remove punctuation and convert to lowercase
    stemmed_tokens = [porter.stem(token) for token in tokens]
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in tokens]
    return tokens, stemmed_tokens, lemmatized_tokens

def get_wordnet_pos(token):
    tag = nltk.pos_tag([token])[0][1][0].upper()  # Get the POS tag
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)  # Return NOUN if not found

combined_df['Tokens'], combined_df['Stemmed'], combined_df['Lemmatized'] = zip(*combined_df['NLP'].apply(process_text))

def check_keywords(tokens):
    cancerlist = []
    for token in tokens:
        if token in unique_list:
            cancerlist.append(token)
    return cancerlist

In [ ]:
df_cleaned = combined_df

df_cleaned['Keywords_Present'] = df_cleaned['Tokens'].apply(check_keywords)
df_cleaned['Keywords_Present2'] = df_cleaned['Stemmed'].apply(check_keywords)
df_cleaned['Keywords_Present3'] = df_cleaned['Lemmatized'].apply(check_keywords)

def is_empty(lst):
    if len(lst) == 0:
        return 0
    else:
        return 1


df_cleaned['is_non_empty'] = df_cleaned['Keywords_Present'].apply(is_empty)
df_cleaned['is_non_empty2'] = df_cleaned['Keywords_Present2'].apply(is_empty)
df_cleaned['is_non_empty3'] = df_cleaned['Keywords_Present3'].apply(is_empty)

df_cleaned['Cancer'] = df_cleaned['is_non_empty'] + df_cleaned['is_non_empty2'] + df_cleaned['is_non_empty3'] #+ df_cleaned['is_non_empty4']
columns_to_drop = ['is_non_empty', 'is_non_empty2','is_non_empty3','Tokens','Stemmed','Lemmatized']
df_cleaned = df_cleaned.drop(columns=columns_to_drop)



In [ ]:

group_sizes = df_cleaned.groupby('Codes')['Cancer'].sum()


group_sizes_df = group_sizes.reset_index(name='count')




In [ ]:
filtered_df = df_cleaned[(df_cleaned['Codes'] == "1140")]
#for i in filtered_df['NLP']:
#    print(i)
    
Uniq_code = list(set(df_cleaned['Codes']))
len(Uniq_code)

603

In [ ]:
for i in Uniq_code:
    filtered_df = df_cleaned[(df_cleaned['Codes'] == i)]
    break

In [ ]:
group_sizes_df.to_excel('Output/Stats.xlsx', index=False)

In [ ]:
group_sizes_df

,Codes,count
0,10,33
1,1000,0
2,1001,0
3,1002,45
4,1003,0
...,...,...
598,996,114
599,997,0
600,998,0
601,999,0


In [ ]:
# Specify the path for the new Excel file
output_excel_path = 'Output/OCR_excel.xlsx'

# Write the combined dataframe to a new Excel file
df_cleaned.to_excel(output_excel_path, index=False)

# Display a message indicating the success
print(f"Combined data has been written to {output_excel_path}")


Combined data has been written to Output/OCR_excel.xlsx
